# **Supervised Fine-Tuning (SFT)**

Supervised Fine-Tuning (SFT) is the process of further training a pre-trained model using a labeled dataset, where the input-output pairs (e.g., article → summary) are known.



Here we have done **full fine-tuning**, meaning all layers of the BART model were updated during training.



**Full fine-tuning** means updating all the parameters (weights) of a pre-trained model on your new dataset. This gives the model full flexibility to adapt to your specific task, like summarization or classification. However, because large models like BART or T5 have hundreds of millions of parameters, full fine-tuning is very resource-intensive — it needs a lot of GPU memory, time, and storage. It’s best used when you have plenty of data and computing power.

**PEFT (Parameter-Efficient Fine-Tuning)** is a smarter, lighter alternative where you freeze most of the model and train only a small number of added parameters, like **LoRA** adapters. This keeps the original model mostly unchanged but still allows it to learn new tasks efficiently. PEFT uses far less memory and is much faster to train, making it ideal for fine-tuning large models on limited hardware or smaller datasets.

# **Full Fine Tuning of BART LLM**

In [ ]:
#Install Required Libraries

!pip install datasets
!pip install transformers

#You install the datasets and transformers libraries needed for loading datasets and models.

In [ ]:
#Load Dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

#You load the CNN/DailyMail summarization dataset, which has article and highlights fields (text + summaries).

In [ ]:
# Load Pre-trained Model and Tokenizer

model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


#You use the BART model, which is powerful for sequence-to-sequence tasks like summarization.

In [ ]:
#Preprocess the Dataset

def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]  # Input text
    targets = [summary for summary in examples["highlights"]]  # Summary
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize targets (summary) with the same settings
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

#You tokenize both the article (input) and highlights (target).
#You pad/truncate both to fixed lengths.
#You return inputs + labels for training.

In [ ]:
# Apply tokenization
tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [ ]:
#Set Training Arguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,  # 🚨 Ensure this is set to False
    report_to="none"  # 🚨 Disable logging to Hugging Face (prevents API key prompt)
)



In [ ]:
#Initialize the Model

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

# You use Hugging Face’s Trainer API.
# You fine-tune the BART model on the training set and validate on the validation set.

In [ ]:
# Save the Fine-tuned Model

model.save_pretrained("./fine_tuned_bart")
tokenizer.save_pretrained("./fine_tuned_bart")

In [ ]:
#Load Fine-tuned Model for Inference

fine_tuned_pipe = pipeline("summarization", model="./fine_tuned_bart")
summary = fine_tuned_pipe(text, max_length=100, min_length=30)

# Load the model using Hugging Face’s pipeline.
# Run a sample text through it to generate a summary.

